In [ ]:
!pip install "jax[cuda]" -f https://storage.googleapis.com/jax-releases/jax_releases.html  # Note: wheels only available on linux.

In [ ]:
!pip install sentencepiece==0.1.96 \
             huggingface-hub==0.4.0 \
             flax==0.4.0 \
             transformers==4.16.1 \
             datasets==1.18.2

In [3]:
import json
import sys
import os
import pandas as pd

In [4]:
# Check gpu availability
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

gpu


In [6]:
if 'COLAB_GPU' in os.environ:
    output_dir = './models'
    train_file = 'full-text-agg-dataset.tsv'
    print('Colab mode')
else:
    output_dir = ''
    train_file = '/Users/naltukhov/Documents/programs/jokes-generator/data/full-text-agg-dataset/full-text-agg-dataset.tsv'
    print('Local mode')

os.makedirs(output_dir, exist_ok=True)
assert os.path.exists(train_file)
print('Input file shape: ', pd.read_csv(train_file, sep='\t').shape)

Colab mode
Input file shape:  (178869, 1)


### Retrain t5 with span mask

In [7]:
params = {
    'output_dir': output_dir,
    'overwrite_output_dir': True,
    'do_train': True,
    'do_eval': True,
    'per_device_train_batch_size': 4,
    'per_device_eval_batch_size': 16,
    'learning_rate': 0.005,
    'weight_decay': 0,
    'adam_beta1': 0.9,
    'adam_beta2': 0.99,
    'adam_epsilon': 1e-08,
    'adafactor': False,
    'num_train_epochs': 10,
    'warmup_steps': 0,
    'logging_steps': 100,
    'save_steps': 100,
    'eval_steps': 200,
    'seed': 5,
    'push_to_hub': False,
    'hub_model_id': None,
    'hub_token': None,
    'model_name_or_path': 'sberbank-ai/ruT5-base',
    'model_type': None,
    'config_name': None,
    'tokenizer_name': None,
    'cache_dir': None,
    'use_fast_tokenizer': False,
    'dtype': 'float32',
    'dataset_name': None,
    'dataset_config_name': None,
    'train_file': train_file,
    'validation_file': None,
    'train_ref_file': None,
    'validation_ref_file': None,
    'overwrite_cache': True,
    'validation_split_percentage': 10,
    'max_seq_length': 1000,
    'preprocessing_num_workers': 4,
    'mlm_probability': 0.15,
    'mean_noise_span_length': 3
}
json.dump(params, open('config_span_mask.json', 'w'))
print('Config saved')

Config saved


In [ ]:
%run train_t5_span_mask.py config_span_mask.json

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-9e32e0f0d9f85cbd/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/980k [00:00<?, ?B/s]